In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


In [2]:


def get_character_info(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    nome = soup.select(
        'h2.pi-item.pi-item-spacing.pi-title.pi-secondary-background')[0].text
    columns = soup.select('h3.pi-data-label.pi-secondary-font')
    # columns = list(filter(lambda x: x.text in ['Nome', 'Nascimento', 'Estado sanguíneo', 'Estado civil', 'Nacionalidade', 'Outros nomes', 'Título(s)', 'Assinatura', 'Espécie', 'Gênero', 'Altura', 'Peso', 'Cor do cabelo', 'Cor dos olhos', 'Cor da pele', 'Membros da família', 'Romances', 'Bicho-papão', 'Varinha', 'Patrono', 'Ocupação', 'Casa', 'Lealdade'], columns)
    infos=[]
    for el in soup.select('div.pi-data-value.pi-font'):

        # if its a list then put a comma between each element
        if len(el.select('li')) > 0:
            infos.append(', '.join([li.text for li in el.select('li')]))
        else:
            infos.append(el.text)

    result = pd.DataFrame([infos], columns=[el.text for el in columns])
    # include name
    result = result.assign(Nome=nome)
    
    return result


In [3]:
sirius = get_character_info(
    'https://harrypotter.fandom.com/pt-br/wiki/Sirius_Black')

hermione = get_character_info(
    'https://harrypotter.fandom.com/pt-br/wiki/Hermione_Granger')


In [4]:
pd.concat([sirius, hermione])

,Nascimento,Morte,Estado sanguíneo,Estado civil,Nacionalidade,Outros nomes,Assinatura,Espécie,Gênero,Altura,...,Bicho-papão,Varinha,Patrono,Casa,Lealdade,Nome,Título(s),Peso,Romances,Ocupação
0,"3 de novembro[1] de 1959[2]Islington, Londres,...","18 de junho de 1996[4] (36 anos), Câmara da Mo...",Sangue puro,Solteiro,Inglês,"Almofadinhas (pelos Marotos e pelo trio)[6], S...",,Humana[9],Masculino[9],"1,75[3]",...,Lorde Voldemort[13],"Comprimento, madeira e núcleo desconhecidos[9]",Cão (possivelmente) [14],Grifinória,"Casa dos Black (anteriormente; deserdado), Esc...",Sirius Black III,NaN,NaN,NaN,NaN
0,"19 de setembro de 1979[1], Inglaterra[2], Grã-...",NaN,Nascida trouxa[3],Casada[4][5],Inglesa,"Hermy (por Grope)[6][7][8], Srta. Perfeição (p...",,Humana[17],Feminino[17],"1,65cm[2]",...,Falhar [23],"Videira, fibra de coração de dragão e vinte e ...",Lontra[24][25],Grifinória[27],"Família Granger, Família Weasley, Família Pott...",Hermione Jean Granger,"Monitora[15], Vice-chefe do Departamento de Ex...",53kg[2],"Vítor Krum (anteriormente), Córmaco McLaggen (...",Funcionária do Departamento para Regulamentaçã...


In [5]:
url_livro = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_a_Pedra_Filosofal'
url_personagem_base = 'https://harrypotter.fandom.com'


def have_banner(href):

    try:
        response = requests.get(url_personagem_base + href)
    except:
        return False

    soup = BeautifulSoup(response.text, 'html.parser')

    columns = soup.select('h3.pi-data-label.pi-secondary-font')
    if 'Nascimento' in [c.text for c in columns]:
        return True
    else:
        return False


def have_informacoes_bibliograficas(href):
    try:
        response = requests.get(url_personagem_base + href)
    except:
        return False

    soup = BeautifulSoup(response.text, 'html.parser')

    columns = soup.select(
        'h2.pi-item.pi-header.pi-secondary-font.pi-item-spacing.pi-secondary-background > center')

    if 'Informações biográficas' in [c.text for c in columns]:
        return True
    else:
        return False


def get_book_info(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # get all hrefs inside paragraphs <p> that have a <a> inside

    links_personagens = []

    # for p in tqdm(soup.select('p')):
    for a in tqdm(soup.select('div.mw-parser-output > p > a')):
        # print(a.text, a['href'])
        # see if the link is a character
        if have_banner(a['href']) or have_informacoes_bibliograficas(a['href']) and (url_personagem_base + a['href']) not in links_personagens:
            # print('achou', a['href'])
            links_personagens.append(a['href'])

    return list(set([url_personagem_base + link for link in links_personagens]))


In [ ]:


href_personagens = get_book_info(url_livro)


In [21]:
url_livro1 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_a_Pedra_Filosofal'
url_livro2 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_a_C%C3%A2mara_Secreta'
url_livro3 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_o_Prisioneiro_de_Azkaban'
url_livro4 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_o_C%C3%A1lice_de_Fogo'
url_livro5 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_a_Ordem_da_F%C3%AAnix'
url_livro6 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_o_Enigma_do_Pr%C3%ADncipe'
url_livro7 = 'https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter_e_as_Rel%C3%ADquias_da_Morte'

livros = [url_livro1, url_livro2, url_livro3,
          url_livro4, url_livro5, url_livro6, url_livro7]

href_personagens = []
for livro in tqdm(livros):
    href_personagens += get_book_info(livro)


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/254 [00:00<?, ?it/s]

  0%|          | 0/264 [00:00<?, ?it/s]

  0%|          | 0/169 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/328 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# with open('href_personagens.txt', 'w') as f:
#     for href in href_personagens:
#         f.write(href + '\n')

In [8]:
# read

with open('href_personagens.txt', 'r') as f:
    # remove \n
    href_personagens = [line[:-1] for line in f.readlines()]

In [10]:
dataframes_personagem = []
for link_personagem in href_personagens:
    print(link_personagem)

    try:
        dataframes_personagem.append(get_character_info(link_personagem))

    except:
        print('error')
        continue



df_personagens = pd.concat(dataframes_personagem)

https://harrypotter.fandom.com/pt-br/wiki/R%C3%BAbeo_Hagrid
https://harrypotter.fandom.com/pt-br/wiki/Irma_Pince
https://harrypotter.fandom.com/pt-br/wiki/F%C3%ADlio_Flitwick
https://harrypotter.fandom.com/pt-br/wiki/Harry_Potter
https://harrypotter.fandom.com/pt-br/wiki/Edwiges
https://harrypotter.fandom.com/pt-br/wiki/Madame_Nor-r-ra
https://harrypotter.fandom.com/pt-br/wiki/J._K._Rowling
https://harrypotter.fandom.com/pt-br/wiki/Pet%C3%BAnia_Dursley
https://harrypotter.fandom.com/pt-br/wiki/Arabella_Figg
https://harrypotter.fandom.com/pt-br/wiki/Grampo
https://harrypotter.fandom.com/pt-br/wiki/Draco_Malfoy
https://harrypotter.fandom.com/pt-br/wiki/Neville_Longbottom
https://harrypotter.fandom.com/pt-br/wiki/Quirino_Quirrell
https://harrypotter.fandom.com/pt-br/wiki/Ronald_Weasley
https://harrypotter.fandom.com/pt-br/wiki/Corn%C3%A9lio_Fudge
https://harrypotter.fandom.com/pt-br/wiki/Severo_Snape
https://harrypotter.fandom.com/pt-br/wiki/Vicente_Crabbe
https://harrypotter.fandom.com/p

In [13]:
(df_personagens
 .drop_duplicates(subset='Nome')
 #drop Joanne Rowling
 .query('Nome != "Joanne Rowling"')
).to_csv('personagens.csv', index=False)

,Nascimento,Estado sanguíneo,Estado civil,Nacionalidade,Outros nomes,Título(s),Assinatura,Espécie,Gênero,Altura,...,Nome,Cor da pele,Romances,Morte,Cor das penas,Dono (s),Forma animaga,Cor do pelo,Título (s),Familiares
0,"6 de dezembro de 1928, Floresta do Deão[1]Sudo...",Parte humano (Meio-gigante),Solteiro[2],Inglês[1],"Hagger (por Grope)[3][4], Hagrid (pelo Trio e ...","Professor, Guardião das Chaves e das Terras de...",,Meio-Humano/Meio-Gigante,Masculino,"3,50cm[5]",...,Rúbeo Hagrid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Anterior a 1966 (provavelmente)[1],NaN,Solteira (provavelmente)[2],Britânica ou irlandesa,NaN,"Madame[3], Bibliotecária[3]",NaN,Humana[3],Feminino[3],NaN,...,Irma Pince,Cor de pergaminho[4],Argo Filch[5],NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,17 de outubro[1][2] de 1958 ou anteriormente[3],Meio-Duende[4],NaN,Britânico ou irlandês[5][6],NaN,"Professor, Doutor[7], Mestre de Feitiços, Maes...",NaN,Humana[10](ancestralidade Duende),Masculino[10],NaN,...,Filio Flitwick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"31 de julho de 1980[1], Godric's Hollow, Sudoe...",Mestiço[2],Casado [3][4],Inglês,"O Menino que Sobreviveu [5][6][7][8], O Eleito...","Campeão Tribruxo, Apanhador, Capitão de Quadri...",,Humana[5],Masculino,NaN,...,Harry Tiago Potter,Clara,"Cho Chang (ex-namorada), Gina Weasley (esposa)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Antes de 1991,NaN,NaN,NaN,NaN,NaN,NaN,Coruja-das-neves,Feminino,NaN,...,Edwiges,NaN,NaN,27 de julho de 1997Inglaterra,Brancas,Harry Potter,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,NaN,NaN,britânico[2],NaN,NaN,NaN,Humana[2],Feminino[2],NaN,...,Dorcas Meadowes,Escura[3],NaN,1981[1],NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,Sangue puro,NaN,Inglês,NaN,NaN,NaN,Humana,Masculino,NaN,...,Fábio Prewett,NaN,NaN,1981[1],NaN,NaN,NaN,NaN,NaN,NaN
0,1925,Puro-sangue[1],Viúva,NaN,NaN,NaN,NaN,Humana[1],Feminino[1],NaN,...,Walburga Black,NaN,NaN,1985[1] (60 anos),NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,Britânico,NaN,NaN,NaN,Humano,Masculino,NaN,...,Benjy Fenwick,NaN,NaN,1981[1],NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
with open('links_personagens.txt', 'w') as f:
    f.write('\n'.join(href_personagens))


In [ ]:
for personagem in href_personagens:
    print(get_character_info(personagem))
    break
